<a href="https://colab.research.google.com/github/navyakarna/PYTORCH_CV/blob/main/GPA_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd


In [3]:
data = pd.read_csv('/content/Prodigy University Dataset.csv')
data.head()

,sat_sum,hs_gpa,fy_gpa
0,508,3.40,3.18
1,488,4.00,3.33
2,464,3.75,3.25
3,380,3.75,2.42
4,428,4.00,2.63


DATA PRE PROCESSING

In [4]:
X = data[['sat_sum', 'hs_gpa']].values #converting data to numpy
#reshape the fy_gpa into a 2D array with [data_size] rows and 1 column
y = data['fy_gpa'].values.reshape(-1, 1)
print(X.shape)
print(y.shape) #we basically have x as input and y as an output

(1000, 2)
(1000, 1)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test , Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train.shape

(800, 2)

In [8]:
import torch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

BUIDLING THE MODEL

In [9]:
import torch.nn as nn


In [10]:
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(), #intoduces non-linearity
    nn.Linear(2, 1)

) #its like an container

In [11]:
preds = model(X_train_tensor) #forward propagation , we basically passed out tensors in the model container

In [12]:
preds[:5] #predictions of first 5 records

tensor([[-0.1089],
        [-0.0988],
        [ 0.0052],
        [-0.0913],
        [-0.0098]], grad_fn=<SliceBackward0>)

In [13]:
from torch.nn import MSELoss # computing the loss

In [14]:
criteron = MSELoss()
loss = criteron(preds, Y_train_tensor)
print(loss)

tensor(6.8966, grad_fn=<MseLossBackward0>)


In [15]:
preds[:5]
Y_train_tensor[:5]

tensor([[2.0000],
        [3.1100],
        [1.6300],
        [3.0200],
        [1.5500]])

In [16]:
model[0].weight

Parameter containing:
tensor([[ 0.3833,  0.0166],
        [-0.4494,  0.5902]], requires_grad=True)

In [17]:
model[2].weight

Parameter containing:
tensor([[-0.3961, -0.5162]], requires_grad=True)

In [18]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.001)

In [19]:
loss.backward()

In [20]:
optimizer.step()

In [21]:
model[0].weight

Parameter containing:
tensor([[ 0.3832,  0.0166],
        [-0.4495,  0.5901]], requires_grad=True)

In [22]:
model[2].weight

Parameter containing:
tensor([[-0.3933, -0.5132]], requires_grad=True)

In [23]:
from torch.utils.data import TensorDataset, DataLoader #data loader shuffle and etc

In [24]:
train_data = TensorDataset(X_train_tensor, Y_train_tensor)

In [26]:
model = nn.Sequential(
    nn.Linear(2,2),
    nn.Sigmoid(),
    nn.Linear(2,1)
)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov =True)


In [27]:
train_loss = criteron(model(X_train_tensor), Y_train_tensor).item()

test_loss = criteron(model(X_test_tensor), Y_test_tensor).item()

print(f'Train Loss: {train_loss:.4f}')
print(f'Test Loss: {test_loss:.4f}')

Train Loss: 9.7272
Test Loss: 10.0661


In [28]:
train_loader = DataLoader(train_data, batch_size=800, shuffle=True) #800 is the number of samples in train set
# Execute the training loop
for epoch in range(1000): # increasing the epochs for effective training
    for X_batch, y_batch in train_loader:
        # Forward pass
        pred = model(X_batch)
        # Changed 'criterion' to 'criteron' to match the previously defined variable
        loss = criteron(pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0: # printing after every 100 epochs
        train_loss = criteron(model(X_train_tensor), Y_train_tensor).item()
        # print(epoch,': ', train_loss)
        # Changed 'criterion' to 'criteron' to match the previously defined variable
        test_loss = criteron(model(X_test_tensor), Y_test_tensor).item()
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch 100: Train Loss: 0.4258, Test Loss: 0.4718
Epoch 200: Train Loss: 0.3997, Test Loss: 0.4465
Epoch 300: Train Loss: 0.3824, Test Loss: 0.4300
Epoch 400: Train Loss: 0.3709, Test Loss: 0.4195
Epoch 500: Train Loss: 0.3633, Test Loss: 0.4128
Epoch 600: Train Loss: 0.3583, Test Loss: 0.4086
Epoch 700: Train Loss: 0.3549, Test Loss: 0.4060
Epoch 800: Train Loss: 0.3527, Test Loss: 0.4044
Epoch 900: Train Loss: 0.3512, Test Loss: 0.4035
Epoch 1000: Train Loss: 0.3501, Test Loss: 0.4029


In [29]:
model = nn.Sequential(
    nn.Linear(2,2),
    nn.Sigmoid(),
    nn.Linear(2,1)
)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov =True)

In [35]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
for epoch in range(500):
  for x_batch, y_batch in train_loader:
    pred = model(x_batch)
    loss = criteron(pred, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  if (epoch+1) % 50 == 0 :
    train_loss = criteron(model(X_train_tensor), Y_train_tensor).item()
    test_loss = criteron(model(X_test_tensor), Y_test_tensor).item()
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch 50: Train Loss: 0.3401, Test Loss: 0.3991
Epoch 100: Train Loss: 0.3400, Test Loss: 0.3989
Epoch 150: Train Loss: 0.3400, Test Loss: 0.3987
Epoch 200: Train Loss: 0.3399, Test Loss: 0.3989
Epoch 250: Train Loss: 0.3399, Test Loss: 0.3988
Epoch 300: Train Loss: 0.3399, Test Loss: 0.3989
Epoch 350: Train Loss: 0.3399, Test Loss: 0.3988
Epoch 400: Train Loss: 0.3398, Test Loss: 0.3985
Epoch 450: Train Loss: 0.3398, Test Loss: 0.3988
Epoch 500: Train Loss: 0.3398, Test Loss: 0.3986


In [39]:
model = nn.Sequential(
    nn.Linear(2,2),
    nn.Sigmoid(),
    nn.Linear(2,1)
)

optimizer = optim.Adagrad(model.parameters())

In [43]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
for epoch in range(500):
  for X_batch , Y_batch in train_loader:
    pred = model(X_batch)
    loss = criteron(pred, Y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  if (epoch +1) % 50 == 0:
    train_loss = criteron(model(X_train_tensor), Y_train_tensor).item()
    test_loss = criteron(model(X_test_tensor), Y_test_tensor).item()
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')



Epoch 50: Train Loss: 0.3894, Test Loss: 0.4469
Epoch 100: Train Loss: 0.3737, Test Loss: 0.4290
Epoch 150: Train Loss: 0.3659, Test Loss: 0.4201
Epoch 200: Train Loss: 0.3617, Test Loss: 0.4153
Epoch 250: Train Loss: 0.3592, Test Loss: 0.4125
Epoch 300: Train Loss: 0.3576, Test Loss: 0.4107
Epoch 350: Train Loss: 0.3565, Test Loss: 0.4094
Epoch 400: Train Loss: 0.3557, Test Loss: 0.4086
Epoch 450: Train Loss: 0.3551, Test Loss: 0.4079
Epoch 500: Train Loss: 0.3547, Test Loss: 0.4074
